# Inventory Optimiser App (Systems Development Coursework)

**Scenario:** Inventory Optimiser for retail chains, restaurants, and distribution centres.

This notebook is designed to be fully runnable in Google Colab for development, testing, and demonstration, with a deployment plan targeting Google App Engine.


In [ ]:
!pip -q install flask pyjwt passlib[bcrypt] tinydb gradio pytest requests pandas gspread google-auth


### Section A — Imports & Configuration
### Section B — SQL Database (SQLite): Structured Inventory Data
### Section C — NoSQL Database (TinyDB): Logs, Alerts, Supplier Feed
### Section D — Cloud Security: Authentication (Register/Login/Logout)
### Section E — REST API (Flask): Inventory Operations
### Section F — Existing Cloud API Example (Google Sheets Supplier Feed)
### Section G — Google Cloud Function Example (Simulated in Colab)
### Section H — Inventory Optimisation Logic (Low-stock + Reorder)
### Section I — Simple Front-End UI (Gradio)
### Section J — Unit Tests (pytest)
### Section K — GitHub Version Control Notes
### Section L — Transition Plan to Google App Engine


## Section A — Imports & Configuration

This section sets up:
- Paths for the SQL (SQLite) and NoSQL (TinyDB JSON) databases.
- Basic security configuration for JWT authentication.



In [ ]:
import os
import sqlite3
import uuid
import datetime

SQLITE_PATH = "inventory_sqlite.db"

# Create a single connection reused throughout the notebook.
# check_same_thread=False keeps things simple for later Flask use in Colab.
sql_conn = sqlite3.connect(SQLITE_PATH, check_same_thread=False)
sql_conn.row_factory = sqlite3.Row

def sql_exec(query: str, params: tuple = ()):
    cur = sql_conn.cursor()
    cur.execute(query, params)
    sql_conn.commit()
    return cur

print("Configured SQLite DB at:", SQLITE_PATH)


Configured SQLite DB at: inventory_sqlite.db


## Section B — SQL Database (SQLite): Structured Inventory Data

You will:
- Create a SQLite database file in Colab runtime storage.
- Create tables for users, locations, inventory items, and stock movements.
- Insert sample data for retail chains, restaurants, and distribution centres.

In [ ]:
def init_sql_schema():
    sql_exec("""
    CREATE TABLE IF NOT EXISTS users (
      id TEXT PRIMARY KEY,
      username TEXT UNIQUE NOT NULL,
      password_hash TEXT NOT NULL,
      created_at TEXT NOT NULL
    );
    """)

    sql_exec("""
    CREATE TABLE IF NOT EXISTS locations (
      id TEXT PRIMARY KEY,
      name TEXT NOT NULL,
      location_type TEXT NOT NULL
        CHECK(location_type IN ('retail_store','restaurant','distribution_centre'))
    );
    """)

    sql_exec("""
    CREATE TABLE IF NOT EXISTS inventory_items (
      id TEXT PRIMARY KEY,
      sku TEXT NOT NULL,
      name TEXT NOT NULL,
      category TEXT NOT NULL,
      unit TEXT NOT NULL,
      location_id TEXT NOT NULL,
      qty_on_hand REAL NOT NULL,
      reorder_point REAL NOT NULL,
      reorder_qty REAL NOT NULL,
      updated_at TEXT NOT NULL,
      FOREIGN KEY(location_id) REFERENCES locations(id)
    );
    """)

    sql_exec("""
    CREATE TABLE IF NOT EXISTS stock_movements (
      id TEXT PRIMARY KEY,
      item_id TEXT NOT NULL,
      change_qty REAL NOT NULL,
      reason TEXT NOT NULL,
      created_at TEXT NOT NULL,
      FOREIGN KEY(item_id) REFERENCES inventory_items(id)
    );
    """)

init_sql_schema()

tables = sql_exec("""
SELECT name FROM sqlite_master
WHERE type='table'
ORDER BY name;
""").fetchall()

print("SQLite tables created:")
for t in tables:
    print("-", t["name"])


SQLite tables created:
- inventory_items
- locations
- stock_movements
- users


In [ ]:
def create_location(name: str, location_type: str) -> str:
    loc_id = str(uuid.uuid4())
    sql_exec(
        "INSERT INTO locations (id, name, location_type) VALUES (?, ?, ?)",
        (loc_id, name, location_type)
    )
    return loc_id

def upsert_item(sku: str, name: str, category: str, unit: str, location_id: str,
                qty_on_hand: float, reorder_point: float, reorder_qty: float) -> str:
    now = datetime.datetime.utcnow().isoformat()
    item_id = f"{location_id}:{sku}"

    existing = sql_exec("SELECT id FROM inventory_items WHERE id = ?", (item_id,)).fetchone()
    if existing:
        sql_exec("""
          UPDATE inventory_items
          SET name=?, category=?, unit=?, qty_on_hand=?, reorder_point=?, reorder_qty=?, updated_at=?
          WHERE id=?
        """, (name, category, unit, qty_on_hand, reorder_point, reorder_qty, now, item_id))
    else:
        sql_exec("""
          INSERT INTO inventory_items (id, sku, name, category, unit, location_id, qty_on_hand, reorder_point, reorder_qty, updated_at)
          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (item_id, sku, name, category, unit, location_id, qty_on_hand, reorder_point, reorder_qty, now))

    return item_id


store1 = create_location("Retail Store - Croydon", "retail_store")
store2 = create_location("Retail Store - Redhill", "retail_store")
rest1  = create_location("Restaurant - Kitchen A", "restaurant")
dc1    = create_location("DC - South Hub", "distribution_centre")


milk_store1 = upsert_item("MILK1L", "Whole Milk 1L", "grocery", "each", store1, qty_on_hand=20, reorder_point=10, reorder_qty=30)
milk_store2 = upsert_item("MILK1L", "Whole Milk 1L", "grocery", "each", store2, qty_on_hand=5,  reorder_point=10, reorder_qty=40)
tomato_rest = upsert_item("TOMATO", "Tomatoes",      "ingredient", "kg", rest1, qty_on_hand=3, reorder_point=5, reorder_qty=8)
rice_dc     = upsert_item("RICE25", "Rice Sack 25kg","bulk", "each", dc1, qty_on_hand=50, reorder_point=60, reorder_qty=40)

print("Locations created:")
print("store1 =", store1)
print("store2 =", store2)
print("rest1  =", rest1)
print("dc1    =", dc1)

print("\nSample item IDs created:")
print("milk_store1 =", milk_store1)
print("milk_store2 =", milk_store2)
print("tomato_rest =", tomato_rest)
print("rice_dc     =", rice_dc)


Locations created:
store1 = f0d031e3-02ad-4346-9681-fcff23d2f7e1
store2 = c41b9c5b-d673-484b-a437-76a17c0cb06a
rest1  = 500e0948-a60b-4d21-a7ef-376dec37e83a
dc1    = 20220b5a-92d6-48d0-b656-0813599c858b

Sample item IDs created:
milk_store1 = f0d031e3-02ad-4346-9681-fcff23d2f7e1:MILK1L
milk_store2 = c41b9c5b-d673-484b-a437-76a17c0cb06a:MILK1L
tomato_rest = 500e0948-a60b-4d21-a7ef-376dec37e83a:TOMATO
rice_dc     = 20220b5a-92d6-48d0-b656-0813599c858b:RICE25


/tmp/ipython-input-3170060433.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


In [ ]:
rows = sql_exec("""
SELECT l.location_type, l.name AS location_name,
       i.sku, i.name AS item_name, i.category, i.unit,
       i.qty_on_hand, i.reorder_point, i.reorder_qty
FROM inventory_items i
JOIN locations l ON l.id = i.location_id
ORDER BY l.location_type, l.name, i.sku;
""").fetchall()

print("Inventory items in SQLite:")
for r in rows:
    print(f"- [{r['location_type']}] {r['location_name']} | {r['sku']} {r['item_name']} | qty={r['qty_on_hand']} {r['unit']} | reorder_point={r['reorder_point']} reorder_qty={r['reorder_qty']}")


Inventory items in SQLite:
- [distribution_centre] DC - South Hub | RICE25 Rice Sack 25kg | qty=50.0 each | reorder_point=60.0 reorder_qty=40.0
- [restaurant] Restaurant - Kitchen A | TOMATO Tomatoes | qty=3.0 kg | reorder_point=5.0 reorder_qty=8.0
- [retail_store] Retail Store - Croydon | MILK1L Whole Milk 1L | qty=20.0 each | reorder_point=10.0 reorder_qty=30.0
- [retail_store] Retail Store - Redhill | MILK1L Whole Milk 1L | qty=5.0 each | reorder_point=10.0 reorder_qty=40.0


## Section C — NoSQL Database (TinyDB): Logs, Alerts, Supplier Feed

You will:
- Initialise TinyDB (JSON document store) as the NoSQL database.
- Store semi-structured/unstructured records such as logs and alerts.
- Store supplier feed snapshots as documents.


In [ ]:
from tinydb import TinyDB, Query

TINYDB_PATH = "inventory_nosql.json"
nosql = TinyDB(TINYDB_PATH)

# Create separate collections (tables) inside TinyDB
events_tbl   = nosql.table("events")         # unstructured logs/events
alerts_tbl   = nosql.table("alerts")         # semi-structured alerts (low-stock, etc.)
supplier_tbl = nosql.table("supplier_feed")  # semi-structured supplier feed snapshots
revoked_tbl  = nosql.table("revoked_tokens") # used later for logout token revocation

print("Configured TinyDB at:", TINYDB_PATH)
print("TinyDB tables:", [t for t in nosql.tables()])

Configured TinyDB at: inventory_nosql.json
TinyDB tables: ['supplier_feed', 'events', 'alerts']


In [ ]:
import datetime
import uuid

now = datetime.datetime.utcnow().isoformat()

# Unstructured event log (free-form fields can vary per event)
events_tbl.insert({
    "ts": now,
    "type": "system.start",
    "message": "Notebook runtime initialised",
    "meta": {"session_id": str(uuid.uuid4())}
})

# Semi-structured alert (consistent core fields, but can evolve with new fields)
alerts_tbl.insert({
    "ts": now,
    "type": "low_stock",
    "severity": "warning",
    "item_id": "DEMO_ITEM",
    "details": {
        "sku": "MILK1L",
        "location_context": "retail_store",
        "qty_on_hand": 5,
        "reorder_point": 10
    }
})

# Semi-structured supplier feed snapshot (shape may differ by supplier)
supplier_tbl.insert({
    "ts": now,
    "source": "demo_supplier_feed",
    "supplier": {"name": "Example Supplier Ltd", "region": "UK"},
    "payload": [
        {"sku": "MILK1L", "lead_time_days": 2, "price": 1.25},
        {"sku": "TOMATO", "lead_time_days": 1, "price": 2.10}
    ]
})

print("Inserted NoSQL records:")
print("- events count  :", len(events_tbl))
print("- alerts count  :", len(alerts_tbl))
print("- supplier count:", len(supplier_tbl))


Inserted NoSQL records:
- events count  : 2
- alerts count  : 2
- supplier count: 2


/tmp/ipython-input-1970261533.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


In [ ]:
# Section C — Query examples (NoSQL)

Alert = Query()
low_stock_alerts = alerts_tbl.search(Alert.type == "low_stock")

print("Low-stock alerts (TinyDB):")
for a in low_stock_alerts:
    print("-", a)

Low-stock alerts (TinyDB):
- {'ts': '2026-01-22T11:15:44.985160', 'type': 'low_stock', 'severity': 'warning', 'item_id': 'DEMO_ITEM', 'details': {'sku': 'MILK1L', 'location_context': 'retail_store', 'qty_on_hand': 5, 'reorder_point': 10}}
- {'ts': '2026-01-22T11:15:52.723322', 'type': 'low_stock', 'severity': 'warning', 'item_id': 'DEMO_ITEM', 'details': {'sku': 'MILK1L', 'location_context': 'retail_store', 'qty_on_hand': 5, 'reorder_point': 10}}


### How this NoSQL data differs from SQL

- **SQLite (SQL)** stores fixed-schema, structured rows (e.g., inventory items with consistent columns and types).
- **TinyDB (NoSQL document store)** stores flexible JSON documents where fields can vary per record (useful for logs, alerts, and supplier feeds that evolve over time).




## Section D - Cloud Security: Authentication(Register/Login/Logout)

This will implement:
*   Secure password hashing(bcrypt)
*   Login to receive a JWT
*   Logout by revoking tokens(blacklist).









In [ ]:
!pip -q install --upgrade "passlib==1.7.4" "bcrypt==3.2.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
from passlib.hash import bcrypt
import jwt

JWT_SECRET = os.getenv("JWT_SECRET", "DEV_ONLY_CHANGE_ME")
JWT_ISSUER = "inventory-optimiser"
JWT_EXP_MINUTES = int(os.getenv("JWT_EXP_MINUTES", "60"))

def register_user(username: str, password: str):
    user_id = str(uuid.uuid4())
    password_hash = bcrypt.hash(password[:72])  # secure hashing, truncate password to 72 bytes

    now = datetime.datetime.utcnow().isoformat()
    try:
        sql_exec(
            "INSERT INTO users (id, username, password_hash, created_at) VALUES (?, ?, ?, ?)",
            (user_id, username, password_hash, now)
        )
    except sqlite3.IntegrityError:
        return {"ok": False, "error": "Username already exists"}

    # Log event (NoSQL, unstructured)
    events_tbl.insert({"ts": now, "type": "auth.register", "username": username})
    return {"ok": True, "user_id": user_id}

def _issue_token(user_id: str, username: str) -> str:
    now = datetime.datetime.utcnow()
    payload = {
        "sub": user_id,
        "username": username,
        "iss": JWT_ISSUER,
        "iat": int(now.timestamp()),
        "exp": int((now + datetime.timedelta(minutes=JWT_EXP_MINUTES)).timestamp()),
        "jti": str(uuid.uuid4())  # unique token id to support logout revocation
    }
    return jwt.encode(payload, JWT_SECRET, algorithm="HS256")

def login_user(username: str, password: str):
    row = sql_exec("SELECT * FROM users WHERE username = ?", (username,)).fetchone()
    if not row:
        return {"ok": False, "error": "Invalid credentials"}

    if not bcrypt.verify(password[:72], row["password_hash"]): # truncate password for verification as well
        return {"ok": False, "error": "Invalid credentials"}

    token = _issue_token(row["id"], row["username"])
    events_tbl.insert({"ts": datetime.datetime.utcnow().isoformat(), "type": "auth.login", "username": username})
    return {"ok": True, "token": token}

def logout_token(token: str):
    """
    JWTs are normally stateless; to demonstrate logout, we implement token revocation
    using a NoSQL blacklist table (revoked_tokens).
    """
    try:
        payload = jwt.decode(token, JWT_SECRET, algorithms=["HS256"], issuer=JWT_ISSUER)
        revoked_tbl.insert({"jti": payload["jti"], "exp": payload["exp"]})
        events_tbl.insert({"ts": datetime.datetime.utcnow().isoformat(), "type": "auth.logout", "username": payload.get("username")})
        return {"ok": True}
    except Exception:
        return {"ok": False, "error": "Invalid token"}

def require_auth(token: str):
    """
    Validates a request token and rejects revoked tokens.
    Used later to protect API endpoints.
    """
    payload = jwt.decode(token, JWT_SECRET, algorithms=["HS256"], issuer=JWT_ISSUER)
    Rev = Query()
    if revoked_tbl.search(Rev.jti == payload["jti"]):
        raise PermissionError("Token revoked")
    return payload

print("Auth functions loaded: register_user, login_user, logout_token, require_auth")

Auth functions loaded: register_user, login_user, logout_token, require_auth


In [ ]:
# Section D — Demo: register, login, protected check, logout

# 1) Register a demo user (run once; if you re-run it, you'll see "Username already exists")
reg = register_user("demo_user", "StrongPassw0rd!")
print("REGISTER:", reg)

# 2) Login to get a JWT
login = login_user("demo_user", "StrongPassw0rd!")
print("\nLOGIN ok:", login.get("ok"))
token = login.get("token")
print("TOKEN present:", token is not None)

# 3) Validate token (simulates protecting an endpoint)
if token:
    payload = require_auth(token)
    print("\nAUTH payload username:", payload.get("username"))

# 4) Logout (revoke token)
if token:
    out = logout_token(token)
    print("\nLOGOUT:", out)

# 5) Confirm token is now rejected
if token:
    try:
        require_auth(token)
        print("ERROR: token should have been revoked but was accepted")
    except Exception as e:
        print("AFTER LOGOUT: token rejected as expected:", type(e).__name__)


/tmp/ipython-input-3581226654.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


REGISTER: {'ok': True, 'user_id': 'a096960c-e96d-40b4-839f-f26d78ed5fc0'}

LOGIN ok: True
TOKEN present: True

AUTH payload username: demo_user

LOGOUT: {'ok': True}
AFTER LOGOUT: token rejected as expected: PermissionError


/tmp/ipython-input-3581226654.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
/tmp/ipython-input-3581226654.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  events_tbl.insert({"ts": datetime.datetime.utcnow().isoformat(), "type": "auth.login", "username": username})
/tmp/ipython-input-3581226654.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  events_tbl.insert({"ts": datetime.datetime.utcnow().isoformat(), "type": "auth.logout", "username": payload.get("username")})


## Section E - REST API (Flask): Inventory Operations

This will implement custom REST API endpoints for:
*   Register, login, logout
*   Stock Updates
*   Low-stock alerts
* Reorder optimisation






In [ ]:
# Section E — Inventory service functions used by the REST API

def record_stock_change(item_id: str, change_qty: float, reason: str):
    now = datetime.datetime.utcnow().isoformat()

    row = sql_exec(
        "SELECT qty_on_hand, reorder_point, reorder_qty, location_id, sku, name FROM inventory_items WHERE id = ?",
        (item_id,)
    ).fetchone()
    if not row:
        return {"ok": False, "error": "Item not found"}

    new_qty = float(row["qty_on_hand"]) + float(change_qty)
    sql_exec("UPDATE inventory_items SET qty_on_hand = ?, updated_at = ? WHERE id = ?", (new_qty, now, item_id))

    sql_exec(
        "INSERT INTO stock_movements (id, item_id, change_qty, reason, created_at) VALUES (?, ?, ?, ?, ?)",
        (str(uuid.uuid4()), item_id, float(change_qty), reason, now)
    )

    # Semi-structured alert stored in NoSQL when below reorder point
    if new_qty < float(row["reorder_point"]):
        alerts_tbl.insert({
            "ts": now,
            "type": "low_stock",
            "severity": "warning",
            "item_id": item_id,
            "sku": row["sku"],
            "name": row["name"],
            "location_id": row["location_id"],
            "qty_on_hand": new_qty,
            "reorder_point": float(row["reorder_point"]),
            "recommended_reorder_qty": float(row["reorder_qty"])
        })

    # Unstructured event log
    events_tbl.insert({"ts": now, "type": "inventory.change", "item_id": item_id, "change_qty": change_qty, "reason": reason})

    return {"ok": True, "new_qty": new_qty}

def list_low_stock(location_id: str | None = None):
    Alert = Query()
    if location_id:
        return alerts_tbl.search((Alert.type == "low_stock") & (Alert.location_id == location_id))
    return alerts_tbl.search(Alert.type == "low_stock")

def optimise_reorder(item_id: str, avg_daily_demand: float, lead_time_days: float, safety_stock: float):
    """
    Simple reorder point model for demonstration:
    reorder_point = avg_daily_demand * lead_time_days + safety_stock
    reorder_qty = max(0, reorder_point - qty_on_hand)
    """
    row = sql_exec("SELECT qty_on_hand FROM inventory_items WHERE id = ?", (item_id,)).fetchone()
    if not row:
        return {"ok": False, "error": "Item not found"}

    qty_on_hand = float(row["qty_on_hand"])
    reorder_point = avg_daily_demand * lead_time_days + safety_stock
    reorder_qty = max(0.0, reorder_point - qty_on_hand)

    now = datetime.datetime.utcnow().isoformat()
    sql_exec(
        "UPDATE inventory_items SET reorder_point = ?, reorder_qty = ?, updated_at = ? WHERE id = ?",
        (reorder_point, reorder_qty, now, item_id)
    )

    events_tbl.insert({"ts": now, "type": "optimise.reorder", "item_id": item_id})
    return {"ok": True, "reorder_point": reorder_point, "reorder_qty": reorder_qty}

print("Inventory service functions loaded: record_stock_change, list_low_stock, optimise_reorder")


Inventory service functions loaded: record_stock_change, list_low_stock, optimise_reorder


In [ ]:
# Section E — Flask REST API (custom endpoints)

from flask import Flask, request, jsonify

app = Flask(__name__)

def _get_bearer_token(req):
    auth = req.headers.get("Authorization", "")
    if auth.startswith("Bearer "):
        return auth.split(" ", 1)[1].strip()
    return None

@app.post("/api/register")
def api_register():
    data = request.get_json(force=True)
    return jsonify(register_user(data["username"], data["password"]))

@app.post("/api/login")
def api_login():
    data = request.get_json(force=True)
    return jsonify(login_user(data["username"], data["password"]))

@app.post("/api/logout")
def api_logout():
    token = _get_bearer_token(request)
    if not token:
        return jsonify({"ok": False, "error": "Missing token"}), 401
    return jsonify(logout_token(token))

@app.get("/api/inventory/low-stock")
def api_low_stock():
    token = _get_bearer_token(request)
    if not token:
        return jsonify({"ok": False, "error": "Missing token"}), 401
    try:
        require_auth(token)
    except Exception:
        return jsonify({"ok": False, "error": "Unauthorized"}), 401

    location_id = request.args.get("location_id")
    return jsonify({"ok": True, "alerts": list_low_stock(location_id)})

@app.post("/api/inventory/stock-change")
def api_stock_change():
    token = _get_bearer_token(request)
    if not token:
        return jsonify({"ok": False, "error": "Missing token"}), 401
    try:
        require_auth(token)
    except Exception:
        return jsonify({"ok": False, "error": "Unauthorized"}), 401

    data = request.get_json(force=True)
    return jsonify(record_stock_change(data["item_id"], float(data["change_qty"]), data["reason"]))

@app.post("/api/optimise/reorder")
def api_optimise_reorder():
    token = _get_bearer_token(request)
    if not token:
        return jsonify({"ok": False, "error": "Missing token"}), 401
    try:
        require_auth(token)
    except Exception:
        return jsonify({"ok": False, "error": "Unauthorized"}), 401

    data = request.get_json(force=True)
    return jsonify(optimise_reorder(
        data["item_id"],
        float(data["avg_daily_demand"]),
        float(data["lead_time_days"]),
        float(data["safety_stock"])
    ))

print("Flask app created with endpoints: /api/register, /api/login, /api/logout, /api/inventory/low-stock, /api/inventory/stock-change, /api/optimise/reorder")


Flask app created with endpoints: /api/register, /api/login, /api/logout, /api/inventory/low-stock, /api/inventory/stock-change, /api/optimise/reorder


In [ ]:
# Section E — Test REST API endpoints inside Colab

client = app.test_client()

# Register and login (API calls)
r1 = client.post("/api/register", json={"username": "api_user", "password": "ApiPassw0rd!"}).get_json()
print("REGISTER:", r1)

r2 = client.post("/api/login", json={"username": "api_user", "password": "ApiPassw0rd!"}).get_json()
print("LOGIN ok:", r2.get("ok"))
token = r2.get("token")
print("TOKEN present:", token is not None)

headers = {"Authorization": f"Bearer {token}"} if token else {}

# Choose a real item_id from your Step 3 output.
# If you don't remember it, query one item_id now:
sample_item_id = sql_exec("SELECT id FROM inventory_items LIMIT 1").fetchone()["id"]
print("\nUsing sample item_id:", sample_item_id)

# Stock change: simulate sale/waste/transfer
r3 = client.post("/api/inventory/stock-change", headers=headers,
                 json={"item_id": sample_item_id, "change_qty": -5, "reason": "sale"}).get_json()
print("\nSTOCK CHANGE:", r3)

# Low-stock alerts
r4 = client.get("/api/inventory/low-stock", headers=headers).get_json()
print("\nLOW STOCK ALERTS:", r4)

# Optimisation
r5 = client.post("/api/optimise/reorder", headers=headers,
                 json={"item_id": sample_item_id, "avg_daily_demand": 6, "lead_time_days": 2, "safety_stock": 5}).get_json()
print("\nOPTIMISE REORDER:", r5)

/tmp/ipython-input-3581226654.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


REGISTER: {'ok': True, 'user_id': '165cfa16-d7a9-4241-a71e-22206fee41f9'}
LOGIN ok: True
TOKEN present: True

Using sample item_id: 20220b5a-92d6-48d0-b656-0813599c858b:RICE25

STOCK CHANGE: {'new_qty': 45.0, 'ok': True}

LOW STOCK ALERTS: {'alerts': [{'details': {'location_context': 'retail_store', 'qty_on_hand': 5, 'reorder_point': 10, 'sku': 'MILK1L'}, 'item_id': 'DEMO_ITEM', 'severity': 'warning', 'ts': '2026-01-22T11:15:44.985160', 'type': 'low_stock'}, {'details': {'location_context': 'retail_store', 'qty_on_hand': 5, 'reorder_point': 10, 'sku': 'MILK1L'}, 'item_id': 'DEMO_ITEM', 'severity': 'warning', 'ts': '2026-01-22T11:15:52.723322', 'type': 'low_stock'}, {'item_id': '20220b5a-92d6-48d0-b656-0813599c858b:RICE25', 'location_id': '20220b5a-92d6-48d0-b656-0813599c858b', 'name': 'Rice Sack 25kg', 'qty_on_hand': 45.0, 'recommended_reorder_qty': 40.0, 'reorder_point': 60.0, 'severity': 'warning', 'sku': 'RICE25', 'ts': '2026-01-22T11:37:32.820748', 'type': 'low_stock'}], 'ok': True

/tmp/ipython-input-3581226654.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
/tmp/ipython-input-3581226654.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  events_tbl.insert({"ts": datetime.datetime.utcnow().isoformat(), "type": "auth.login", "username": username})
/tmp/ipython-input-165053263.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()
/tmp/ipython-input-165053263.py:61: DeprecationWarning: datetime.datetime.utcnow() is de

In [ ]:
# Step A: Login via the REST API to get a token (Flask test client)

client = app.test_client()

# If this user already exists, register will return ok=False; that's fine.
print(client.post("/api/register", json={"username": "alert_user", "password": "AlertPassw0rd!"}).get_json())

login = client.post("/api/login", json={"username": "alert_user", "password": "AlertPassw0rd!"}).get_json()
print("LOGIN:", {"ok": login.get("ok"), "token_present": bool(login.get("token"))})

token = login.get("token")
headers = {"Authorization": f"Bearer {token}"} if token else {}


/tmp/ipython-input-3581226654.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


{'ok': True, 'user_id': 'ad98f246-b09b-4f77-a247-69358c3f4176'}
LOGIN: {'ok': True, 'token_present': True}


/tmp/ipython-input-3581226654.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
/tmp/ipython-input-3581226654.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  events_tbl.insert({"ts": datetime.datetime.utcnow().isoformat(), "type": "auth.login", "username": username})


In [ ]:
# Step B: Choose an item and reduce stock enough to go below reorder_point

# Pick an item (you can change the WHERE sku='...' to MILK 1L / TOMATO / RICE 25)
row = sql_exec("""
SELECT id, sku, qty_on_hand, reorder_point
FROM inventory_items
WHERE sku = 'TOMATO'
LIMIT 1
""").fetchone()

print("Before:", dict(row))

# Reduce stock to guarantee below reorder_point:
# Make change_qty negative and large enough.
change_qty = -(float(row["qty_on_hand"]) + 1)

resp = client.post(
    "/api/inventory/stock-change",
    headers=headers,
    json={"item_id": row["id"], "change_qty": change_qty, "reason": "waste"}
).get_json()

print("API response:", resp)

after = sql_exec("SELECT id, sku, qty_on_hand, reorder_point FROM inventory_items WHERE id = ?", (row["id"],)).fetchone()
print("After:", dict(after))


Before: {'id': 'a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO', 'sku': 'TOMATO', 'qty_on_hand': 3.0, 'reorder_point': 2.5}
API response: {'new_qty': -1.0, 'ok': True}
After: {'id': 'a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO', 'sku': 'TOMATO', 'qty_on_hand': -1.0, 'reorder_point': 2.5}


/tmp/ipython-input-165053263.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


## Section F - Existing Cloud API



## Section G Goolge Cloud Functions(demostration)

In [ ]:
# Section G — Cloud Function example (HTTP-triggered) - code only, not deployed from Colab

cloud_function_main_py = r'''
import datetime

# In a deployed Cloud Function, you would typically use a managed database (e.g., Firestore)
# and IAM-based credentials rather than local files.
#
# This function is designed to be called by the main app when a low-stock alert occurs.

def low_stock_webhook(request):
    """
    Expected JSON payload:
    {
      "type": "low_stock",
      "item_id": "...",
      "sku": "...",
      "location_id": "...",
      "qty_on_hand": 3,
      "reorder_point": 10,
      "recommended_reorder_qty": 8
    }
    """
    data = request.get_json(silent=True) or {}
    data["received_at"] = datetime.datetime.utcnow().isoformat()

    # For demonstration: return data to confirm receipt.
    # In production: write to Firestore, trigger notifications, or publish to Pub/Sub.
    return ({"ok": True, "received": data}, 200)
'''

with open("gcf_main.py", "w") as f:
    f.write(cloud_function_main_py)

print("Created gcf_main.py (Cloud Function example). First 25 lines:\n")
print("\n".join(cloud_function_main_py.splitlines()[:25]))


Created gcf_main.py (Cloud Function example). First 25 lines:


import datetime

# In a deployed Cloud Function, you would typically use a managed database (e.g., Firestore)
# and IAM-based credentials rather than local files.
#
# This function is designed to be called by the main app when a low-stock alert occurs.

def low_stock_webhook(request):
    """
    Expected JSON payload:
    {
      "type": "low_stock",
      "item_id": "...",
      "sku": "...",
      "location_id": "...",
      "qty_on_hand": 3,
      "reorder_point": 10,
      "recommended_reorder_qty": 8
    }
    """
    data = request.get_json(silent=True) or {}
    data["received_at"] = datetime.datetime.utcnow().isoformat()

    # For demonstration: return data to confirm receipt.


In [ ]:
# Section G — Simulate a Cloud Function invocation in Colab (NOT deployed)

# A tiny fake request object to mimic request.get_json() used by Cloud Functions
class FakeRequest:
    def __init__(self, payload):
        self._payload = payload
    def get_json(self, silent=True):
        return self._payload

# Import the function from the file we wrote
import importlib.util

spec = importlib.util.spec_from_file_location("gcf_main", "gcf_main.py")
gcf_main = importlib.util.module_from_spec(spec)
spec.loader.exec_module(gcf_main)

# Build a realistic low-stock payload (similar to what our app generates in NoSQL)
sample_payload = {
    "type": "low_stock",
    "item_id": sql_exec("SELECT id FROM inventory_items LIMIT 1").fetchone()["id"],
    "sku": sql_exec("SELECT sku FROM inventory_items LIMIT 1").fetchone()["sku"],
    "location_id": sql_exec("SELECT location_id FROM inventory_items LIMIT 1").fetchone()["location_id"],
    "qty_on_hand": 2,
    "reorder_point": 10,
    "recommended_reorder_qty": 8
}

resp, status = gcf_main.low_stock_webhook(FakeRequest(sample_payload))
print("Simulated Cloud Function HTTP status:", status)
print("Simulated Cloud Function response:", resp)


Simulated Cloud Function HTTP status: 200
Simulated Cloud Function response: {'ok': True, 'received': {'type': 'low_stock', 'item_id': '20220b5a-92d6-48d0-b656-0813599c858b:RICE25', 'sku': 'MILK1L', 'location_id': 'e5f96ed9-3456-4bb0-8b6b-59fa8a41c535', 'qty_on_hand': 2, 'reorder_point': 10, 'recommended_reorder_qty': 8, 'received_at': '2026-01-22T11:41:29.000811'}}


## Section H - Inventory Optimisation Logic

In [ ]:
# Section H — Inventory optimisation helpers

def get_item(item_id: str):
    row = sql_exec("""
      SELECT i.*, l.name AS location_name, l.location_type
      FROM inventory_items i
      JOIN locations l ON l.id = i.location_id
      WHERE i.id = ?
    """, (item_id,)).fetchone()
    return row

def compute_reorder_point(avg_daily_demand: float, lead_time_days: float, safety_stock: float) -> float:
    # reorder_point = demand during lead time + safety stock
    return float(avg_daily_demand) * float(lead_time_days) + float(safety_stock)

def compute_reorder_qty(qty_on_hand: float, reorder_point: float) -> float:
    # reorder_qty = how much to buy to reach reorder point (simple demo model)
    return max(0.0, float(reorder_point) - float(qty_on_hand))

print("Optimisation helper functions loaded: get_item, compute_reorder_point, compute_reorder_qty")


Optimisation helper functions loaded: get_item, compute_reorder_point, compute_reorder_qty


In [ ]:
# Section H — Optimise reorder settings + generate alert if needed

def optimise_item_reorder(item_id: str, avg_daily_demand: float, lead_time_days: float, safety_stock: float):
    row = get_item(item_id)
    if not row:
        return {"ok": False, "error": "Item not found"}

    reorder_point = compute_reorder_point(avg_daily_demand, lead_time_days, safety_stock)
    reorder_qty = compute_reorder_qty(row["qty_on_hand"], reorder_point)

    now = datetime.datetime.utcnow().isoformat()
    sql_exec("""
      UPDATE inventory_items
      SET reorder_point = ?, reorder_qty = ?, updated_at = ?
      WHERE id = ?
    """, (reorder_point, reorder_qty, now, item_id))

    # If currently below the new reorder point, store a semi-structured alert in NoSQL
    if float(row["qty_on_hand"]) < reorder_point:
        alerts_tbl.insert({
            "ts": now,
            "type": "low_stock",
            "severity": "warning",
            "item_id": item_id,
            "sku": row["sku"],
            "name": row["name"],
            "location_id": row["location_id"],
            "location_type": row["location_type"],
            "location_name": row["location_name"],
            "qty_on_hand": float(row["qty_on_hand"]),
            "reorder_point": reorder_point,
            "recommended_reorder_qty": reorder_qty,
            "inputs": {
                "avg_daily_demand": float(avg_daily_demand),
                "lead_time_days": float(lead_time_days),
                "safety_stock": float(safety_stock)
            }
        })

    # Unstructured log
    events_tbl.insert({
        "ts": now,
        "type": "optimise.run",
        "item_id": item_id,
        "note": "Reorder settings recalculated"
    })

    return {"ok": True, "item_id": item_id, "reorder_point": reorder_point, "reorder_qty": reorder_qty}

print("Loaded optimise_item_reorder(item_id, avg_daily_demand, lead_time_days, safety_stock)")


Loaded optimise_item_reorder(item_id, avg_daily_demand, lead_time_days, safety_stock)


In [ ]:
# Section H — Run optimisation examples for the 3 contexts

# Grab known item_ids from the database (created in Step 3)
items = sql_exec("""
  SELECT id, sku
  FROM inventory_items
""").fetchall()
item_ids = {r["sku"]: [] for r in items}
for r in items:
    item_ids[r["sku"]].append(r["id"])

print("Available SKUs and item_ids:")
for sku, ids in item_ids.items():
    print("-", sku, "->", ids)

# Choose one item per context by SKU (based on sample data from Step 3)
retail_item_id = item_ids["MILK1L"][0]     # retail chain example (multi-store SKU)
restaurant_item_id = item_ids["TOMATO"][0] # restaurant ingredient example
dc_item_id = item_ids["RICE25"][0]         # distribution centre bulk example

print("\nChosen item IDs:")
print("Retail (MILK1L):", retail_item_id)
print("Restaurant (TOMATO):", restaurant_item_id)
print("Distribution (RICE25):", dc_item_id)

# Retail chains: higher daily demand, moderate lead time
out_retail = optimise_item_reorder(retail_item_id, avg_daily_demand=6, lead_time_days=2, safety_stock=5)
print("\nRetail optimisation output:", out_retail)

# Restaurants: perishable ingredient, short lead time, safety stock tuned to reduce waste
out_rest = optimise_item_reorder(restaurant_item_id, avg_daily_demand=1.5, lead_time_days=1, safety_stock=1)
print("\nRestaurant optimisation output:", out_rest)

# Distribution centres: bulk movement, longer lead time, larger safety stock
out_dc = optimise_item_reorder(dc_item_id, avg_daily_demand=4, lead_time_days=5, safety_stock=10)
print("\nDistribution optimisation output:", out_dc)


Available SKUs and item_ids:
- MILK1L -> ['e5f96ed9-3456-4bb0-8b6b-59fa8a41c535:MILK1L', '498d2072-4e0e-49e2-8dac-e6fd7a14bf03:MILK1L', 'f0d031e3-02ad-4346-9681-fcff23d2f7e1:MILK1L', 'c41b9c5b-d673-484b-a437-76a17c0cb06a:MILK1L']
- TOMATO -> ['a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO', '500e0948-a60b-4d21-a7ef-376dec37e83a:TOMATO']
- RICE25 -> ['3d1893d9-769f-4727-b1ed-1f805244120b:RICE25', '20220b5a-92d6-48d0-b656-0813599c858b:RICE25']

Chosen item IDs:
Retail (MILK1L): e5f96ed9-3456-4bb0-8b6b-59fa8a41c535:MILK1L
Restaurant (TOMATO): a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO
Distribution (RICE25): 3d1893d9-769f-4727-b1ed-1f805244120b:RICE25

Retail optimisation output: {'ok': True, 'item_id': 'e5f96ed9-3456-4bb0-8b6b-59fa8a41c535:MILK1L', 'reorder_point': 17.0, 'reorder_qty': 0.0}

Restaurant optimisation output: {'ok': True, 'item_id': 'a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO', 'reorder_point': 2.5, 'reorder_qty': 0.0}

Distribution optimisation output: {'ok': True, 'item_id

/tmp/ipython-input-2812528324.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat()


In [ ]:
# Section H — View low-stock alerts created by optimisation

Alert = Query()
recent_alerts = alerts_tbl.search(Alert.type == "low_stock")

print("Low-stock alerts in NoSQL (TinyDB):", len(recent_alerts))
for a in recent_alerts[-5:]:
    print("-", a.get("location_type"), "|", a.get("location_name"), "|", a.get("sku"), "| qty:", a.get("qty_on_hand"),
          "| reorder_point:", a.get("reorder_point"), "| suggested:", a.get("recommended_reorder_qty"))


Low-stock alerts in NoSQL (TinyDB): 4
- None | None | None | qty: None | reorder_point: None | suggested: None
- None | None | None | qty: None | reorder_point: None | suggested: None
- None | None | RICE25 | qty: 45.0 | reorder_point: 60.0 | suggested: 40.0
- None | None | TOMATO | qty: -1.0 | reorder_point: 2.5 | suggested: 0.0


In [ ]:
# Diagnostic: confirm alerts exist and show the latest 5 raw documents

print("alerts_tbl exists:", "alerts_tbl" in globals())

try:
    all_alerts = alerts_tbl.all()
    print("Total alerts in TinyDB:", len(all_alerts))
    print("\nLast 5 alert documents (raw):")
    for a in all_alerts[-5:]:
        print(a)
except Exception as e:
    print("ERROR reading alerts_tbl:", type(e).__name__, str(e))


alerts_tbl exists: True
Total alerts in TinyDB: 4

Last 5 alert documents (raw):
{'ts': '2026-01-22T11:15:44.985160', 'type': 'low_stock', 'severity': 'warning', 'item_id': 'DEMO_ITEM', 'details': {'sku': 'MILK1L', 'location_context': 'retail_store', 'qty_on_hand': 5, 'reorder_point': 10}}
{'ts': '2026-01-22T11:15:52.723322', 'type': 'low_stock', 'severity': 'warning', 'item_id': 'DEMO_ITEM', 'details': {'sku': 'MILK1L', 'location_context': 'retail_store', 'qty_on_hand': 5, 'reorder_point': 10}}
{'ts': '2026-01-22T11:37:32.820748', 'type': 'low_stock', 'severity': 'warning', 'item_id': '20220b5a-92d6-48d0-b656-0813599c858b:RICE25', 'sku': 'RICE25', 'name': 'Rice Sack 25kg', 'location_id': '20220b5a-92d6-48d0-b656-0813599c858b', 'qty_on_hand': 45.0, 'reorder_point': 60.0, 'recommended_reorder_qty': 40.0}
{'ts': '2026-01-22T11:55:59.151585', 'type': 'low_stock', 'severity': 'warning', 'item_id': 'a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO', 'sku': 'TOMATO', 'name': 'Tomatoes', 'location

In [ ]:
# Section H — Improved alert viewer (handles different alert shapes + enriches with SQL location data)

from tinydb import Query

def _enrich_location(location_id: str):
    if not location_id:
        return {"location_type": None, "location_name": None}
    row = sql_exec("SELECT location_type, name FROM locations WHERE id = ?", (location_id,)).fetchone()
    if not row:
        return {"location_type": None, "location_name": None}
    return {"location_type": row["location_type"], "location_name": row["name"]}

Alert = Query()
recent_alerts = alerts_tbl.search(Alert.type == "low_stock")

print("Low-stock alerts in NoSQL (TinyDB):", len(recent_alerts))
print("\n--- Latest alerts (up to 10) ---")

for a in recent_alerts[-10:]:
    # Handle both formats:
    # (1) Older demo alert: a["details"]["sku"], etc.
    # (2) App alerts: a["sku"], a["location_id"], etc.
    details = a.get("details", {}) if isinstance(a.get("details", {}), dict) else {}

    sku = a.get("sku") or details.get("sku")
    qty = a.get("qty_on_hand") if a.get("qty_on_hand") is not None else details.get("qty_on_hand")
    rp  = a.get("reorder_point") if a.get("reorder_point") is not None else details.get("reorder_point")

    location_id = a.get("location_id")
    loc = _enrich_location(location_id)

    print(
        f"- ts={a.get('ts')} | sku={sku} | qty={qty} | reorder_point={rp} | "
        f"location_type={a.get('location_type') or loc['location_type'] or details.get('location_context')} | "
        f"location_name={a.get('location_name') or loc['location_name']} | "
        f"item_id={a.get('item_id')}"
    )

Low-stock alerts in NoSQL (TinyDB): 4

--- Latest alerts (up to 10) ---
- ts=2026-01-22T11:15:44.985160 | sku=MILK1L | qty=5 | reorder_point=10 | location_type=retail_store | location_name=None | item_id=DEMO_ITEM
- ts=2026-01-22T11:15:52.723322 | sku=MILK1L | qty=5 | reorder_point=10 | location_type=retail_store | location_name=None | item_id=DEMO_ITEM
- ts=2026-01-22T11:37:32.820748 | sku=RICE25 | qty=45.0 | reorder_point=60.0 | location_type=distribution_centre | location_name=DC - South Hub | item_id=20220b5a-92d6-48d0-b656-0813599c858b:RICE25
- ts=2026-01-22T11:55:59.151585 | sku=TOMATO | qty=-1.0 | reorder_point=2.5 | location_type=restaurant | location_name=Restaurant - Kitchen A | item_id=a0f914bc-a2dd-4e7c-8502-f589808ca2ba:TOMATO


## Section I - Simple Front-end demonstration

In [ ]:
# Section I — Simple Front-End UI (Gradio)

import gradio as gr

def ui_register(username, password):
    return register_user(username, password)

def ui_login(username, password):
    return login_user(username, password)

def ui_stock_change(token, item_id, change_qty, reason):
    try:
        require_auth(token)
    except Exception:
        return {"ok": False, "error": "Unauthorized (invalid/revoked token)"}
    return record_stock_change(item_id, float(change_qty), reason)

def ui_low_stock(token, location_id):
    try:
        require_auth(token)
    except Exception:
        return {"ok": False, "error": "Unauthorized (invalid/revoked token)"}
    return {"ok": True, "alerts": list_low_stock(location_id if location_id else None)}

def ui_list_items():
    rows = sql_exec("""
      SELECT i.id, l.location_type, l.name AS location_name, i.sku, i.name AS item_name, i.qty_on_hand, i.reorder_point
      FROM inventory_items i
      JOIN locations l ON l.id = i.location_id
      ORDER BY l.location_type, l.name, i.sku
    """).fetchall()
    return [
        {
            "item_id": r["id"],
            "location_type": r["location_type"],
            "location_name": r["location_name"],
            "sku": r["sku"],
            "item_name": r["item_name"],
            "qty_on_hand": r["qty_on_hand"],
            "reorder_point": r["reorder_point"]
        }
        for r in rows
    ]

with gr.Blocks() as demo:
    gr.Markdown("# Inventory Optimiser App — Simple UI (Colab)")

    with gr.Tab("Auth"):
        gr.Markdown("### Register / Login (JWT token)")
        u = gr.Textbox(label="Username")
        p = gr.Textbox(label="Password", type="password")

        out_reg = gr.JSON(label="Register output")
        gr.Button("Register").click(ui_register, [u, p], out_reg)

        out_login = gr.JSON(label="Login output (copy token)")
        gr.Button("Login").click(ui_login, [u, p], out_login)

    with gr.Tab("Inventory"):
        gr.Markdown("### Inventory operations (token required)")
        token = gr.Textbox(label="JWT token (paste from Login output)")

        items_out = gr.JSON(label="Current inventory items (copy an item_id)")
        gr.Button("List inventory items").click(ui_list_items, [], items_out)

        item_id = gr.Textbox(label="Item ID (paste from inventory list)")
        change_qty = gr.Number(label="Change qty (negative = sale/waste; positive = delivery)")
        reason = gr.Textbox(label="Reason (sale, waste, delivery, transfer)")

        out_change = gr.JSON(label="Stock change output")
        gr.Button("Record stock change").click(ui_stock_change, [token, item_id, change_qty, reason], out_change)

        location_id = gr.Textbox(label="Location ID (optional)")
        out_alerts = gr.JSON(label="Low-stock alerts")
        gr.Button("View low-stock alerts").click(ui_low_stock, [token, location_id], out_alerts)

demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### How this satisfies the UI requirement

This Gradio interface is a **simple front-end user interface** that allows:
- User registration/login to obtain a token
- Inventory listing and stock updates
- Viewing low-stock alerts

It runs in Google Colab for demonstration and calls the same application logic as the REST API.


## Section J - Unit Testing

In [ ]:
# Section J — Create unit test file (pytest)

%%writefile test_inventory_app.py
import uuid

def test_password_hashing_roundtrip():
    # Import inside test so it uses the same environment as the notebook
    from passlib.hash import bcrypt
    h = bcrypt.hash("abc123!XYZ")
    assert bcrypt.verify("abc123!XYZ", h)
    assert not bcrypt.verify("wrong", h)

def test_reorder_math_helpers():
    # These functions are defined in Section H
    from __main__ import compute_reorder_point, compute_reorder_qty
    rp = compute_reorder_point(avg_daily_demand=6, lead_time_days=2, safety_stock=5)
    assert rp == 17.0
    rq = compute_reorder_qty(qty_on_hand=10, reorder_point=rp)
    assert rq == 7.0

def test_sql_inventory_item_exists():
    # Confirms SQL persistence has at least one inventory item row
    from __main__ import sql_exec
    row = sql_exec("SELECT COUNT(*) AS c FROM inventory_items").fetchone()
    assert row["c"] >= 1

def test_stock_change_updates_quantity():
    # Uses your service function and verifies SQL changes
    from __main__ import sql_exec, record_stock_change

    item_row = sql_exec("SELECT id, qty_on_hand FROM inventory_items LIMIT 1").fetchone()
    item_id = item_row["id"]
    before = float(item_row["qty_on_hand"])

    out = record_stock_change(item_id, change_qty=-1, reason="sale")
    assert out["ok"] is True

    after_row = sql_exec("SELECT qty_on_hand FROM inventory_items WHERE id = ?", (item_id,)).fetchone()
    after = float(after_row["qty_on_hand"])
    assert after == before - 1


Writing test_inventory_app.py


In [ ]:
# STEP 10 FIX-1: Write a small module so pytest can import app logic

%%writefile inventory_app_lib.py
import sqlite3
import uuid
import datetime

SQLITE_PATH = "inventory_sqlite.db"

def sql_exec(query: str, params: tuple = ()):
    conn = sqlite3.connect(SQLITE_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(query, params)
    conn.commit()
    return cur, conn

def compute_reorder_point(avg_daily_demand: float, lead_time_days: float, safety_stock: float) -> float:
    return float(avg_daily_demand) * float(lead_time_days) + float(safety_stock)

def compute_reorder_qty(qty_on_hand: float, reorder_point: float) -> float:
    return max(0.0, float(reorder_point) - float(qty_on_hand))

def record_stock_change(item_id: str, change_qty: float, reason: str):
    now = datetime.datetime.utcnow().isoformat()

    cur, conn = sql_exec(
        "SELECT qty_on_hand FROM inventory_items WHERE id = ?",
        (item_id,)
    )
    row = cur.fetchone()
    if not row:
        conn.close()
        return {"ok": False, "error": "Item not found"}

    new_qty = float(row["qty_on_hand"]) + float(change_qty)

    cur.execute("UPDATE inventory_items SET qty_on_hand = ?, updated_at = ? WHERE id = ?", (new_qty, now, item_id))
    cur.execute(
        "INSERT INTO stock_movements (id, item_id, change_qty, reason, created_at) VALUES (?, ?, ?, ?, ?)",
        (str(uuid.uuid4()), item_id, float(change_qty), reason, now)
    )
    conn.commit()
    conn.close()
    return {"ok": True, "new_qty": new_qty}


Writing inventory_app_lib.py


In [ ]:
# STEP 10 FIX-2: Rewrite tests to import from inventory_app_lib (not __main__)

%%writefile test_inventory_app.py
def test_password_hashing_roundtrip():
    from passlib.hash import bcrypt
    h = bcrypt.hash("abc123!XYZ")
    assert bcrypt.verify("abc123!XYZ", h)
    assert not bcrypt.verify("wrong", h)

def test_reorder_math_helpers():
    from inventory_app_lib import compute_reorder_point, compute_reorder_qty
    rp = compute_reorder_point(avg_daily_demand=6, lead_time_days=2, safety_stock=5)
    assert rp == 17.0
    rq = compute_reorder_qty(qty_on_hand=10, reorder_point=rp)
    assert rq == 7.0

def test_sql_inventory_item_exists():
    from inventory_app_lib import sql_exec
    cur, conn = sql_exec("SELECT COUNT(*) AS c FROM inventory_items")
    row = cur.fetchone()
    conn.close()
    assert row["c"] >= 1

def test_stock_change_updates_quantity():
    from inventory_app_lib import sql_exec, record_stock_change

    cur, conn = sql_exec("SELECT id, qty_on_hand FROM inventory_items LIMIT 1")
    item_row = cur.fetchone()
    conn.close()

    item_id = item_row["id"]
    before = float(item_row["qty_on_hand"])

    out = record_stock_change(item_id, change_qty=-1, reason="sale")
    assert out["ok"] is True

    cur2, conn2 = sql_exec("SELECT qty_on_hand FROM inventory_items WHERE id = ?", (item_id,))
    after_row = cur2.fetchone()
    conn2.close()

    after = float(after_row["qty_on_hand"])
    assert after == before - 1


Overwriting test_inventory_app.py


In [ ]:
# Section J — Run unit tests

!pytest -q

....                                                                     [100%]
4 passed in 1.09s


## Section K - GitHub Version Control

In [ ]:
# Section K — Create an export folder for GitHub

import os, shutil

EXPORT_DIR = "github_export"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Copy key Python artifacts created in Colab
for fname in ["inventory_app_lib.py", "gcf_main.py", "test_inventory_app.py", "inventory_sqlite.db", "inventory_nosql.json"]:
    if os.path.exists(fname):
        shutil.copy(fname, os.path.join(EXPORT_DIR, fname))

print("Created export folder:", EXPORT_DIR)
print("Files currently in export folder:")
print("\n".join(sorted(os.listdir(EXPORT_DIR))) if os.listdir(EXPORT_DIR) else "(empty)")


Created export folder: github_export
Files currently in export folder:
gcf_main.py
inventory_app_lib.py
inventory_nosql.json
inventory_sqlite.db
test_inventory_app.py
